In [1]:
# Create data for integrated cell viz

In [2]:
!date

Tue Feb 22 15:37:17 PST 2022


In [15]:
import os
import sys
import importlib
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

import common

In [4]:
importlib.reload(common)

<module 'common' from '/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/FigureMorphing/common.py'>

In [5]:
# Controller form cvapipe_analysis
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
control = controller.Controller(general.load_config_file(path_config))
print(control.get_staging())

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


### Create the domain to morph structures into

In [6]:
space = shapespace.ShapeSpaceBasic(control)
variables = control.get_variables_values_for_aggregation()
df = space.get_aggregated_df(variables).drop(columns=["structure"]).drop_duplicates().reset_index(drop=True)
df = df.loc[df.mpId==control.get_center_map_point_index()].head(1)
df

,shape_mode,mpId,aggtype,alias
4,NUC_MEM_PC1,5,avg,STR


In [7]:
row = df.loc[df.index[0]]
device = io.LocalStagingIO(control)
device.row = row

In [8]:
nisos = control.get_number_of_interpolating_points()
inner_alias = control.get_inner_most_alias_to_parameterize()
inner_mesh = device.read_map_point_mesh(inner_alias)
outer_alias = control.get_outer_most_alias_to_parameterize()
outer_mesh = device.read_map_point_mesh(outer_alias)
domain, origin = cytoparam.voxelize_meshes([outer_mesh, inner_mesh])
coords_param, coeffs_centroid = cytoparam.parameterize_image_coordinates(
    seg_mem=(domain>0).astype(np.uint8),
    seg_nuc=(domain>1).astype(np.uint8),
    lmax=control.get_lmax(), nisos=[nisos, nisos]
)
coeffs_mem, centroid_mem, coeffs_nuc, centroid_nuc = coeffs_centroid
coords_param += np.array(centroid_nuc).reshape(3, 1, 1)-np.array(centroid_mem).reshape(3, 1, 1)

In [9]:
variables = control.get_variables_values_for_aggregation()
variables.update({"shape_mode": ["NdSphere"], "mpId": [control.get_center_map_point_index()]})
space = shapespace.ShapeSpaceBasic(control)
df = space.get_aggregated_df(variables)
df.head()

,shape_mode,mpId,aggtype,alias,structure
0,NdSphere,5,avg,STR,FBL
1,NdSphere,5,avg,STR,NPM1
2,NdSphere,5,avg,STR,SON
3,NdSphere,5,avg,STR,SMC1A
4,NdSphere,5,avg,STR,HIST1H2BJ


### Morph structures into domain

In [12]:
data = {}
for gene in tqdm(control.get_gene_names()):
    row = df.loc[df.structure==gene].iloc[0]
    rep = device.read_agg_parameterized_intensity(row)
    morphed = cytoparam.morph_representation_on_shape(
        img=domain,
        param_img_coords=coords_param,
        representation=rep
    )
    data[gene] = morphed

  0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
# Output folder
output = "/allen/aics/assay-dev/MicroscopyOtherData/Viana/forGraham/variance/cell_integration/data"

In [17]:
for gene, img in data.items():
    fname = os.path.join(output,f'{gene}_NdSphere.tif')
    OmeTiffWriter.save(img, fname, dim_order="ZYX", overwrite_file=True)